<a href="https://colab.research.google.com/github/ananyabelavadi/Ananya-/blob/main/movie_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Movie Recommendation System using SVD**

In [ ]:
!pip install surprise
!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357249 sha256=d2e09c969399aa46a1f37d655f608a692ae7c4790443b4f9e271bb5eb0b1b10f
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
# Download the MovieLens dataset
!wget -nc https://files.grouplens.org/datasets/movielens/ml-latest-small.zip -O ml-latest-small.zip

# Unzip the dataset
!unzip -o ml-latest-small.zip


--2024-06-18 06:11:04--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2024-06-18 06:11:05 (7.19 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
import pandas as pd

# Load the datasets
movies_data = pd.read_csv('ml-latest-small/movies.csv')
ratings_data = pd.read_csv('ml-latest-small/ratings.csv')

# Display the first few rows of the datasets
print(movies_data.head())
print(ratings_data.head())


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [ ]:
from surprise import Dataset, Reader

# Create a Reader object with the appropriate rating scale
reader = Reader(rating_scale=(0.5, 5.0))

# Load the dataset into Surprise
data = Dataset.load_from_df(ratings_data[['userId', 'movieId', 'rating']], reader)


In [ ]:
from surprise.model_selection import train_test_split

# Split the data into training and test sets
trainset, testset = train_test_split(data, test_size=0.25)


In [ ]:
from surprise import SVD

# Define the SVD algorithm
algo = SVD()

# Train the algorithm on the training set
algo.fit(trainset)


In [ ]:
from surprise.model_selection import cross_validate

# Perform cross-validation
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8716  0.8749  0.8644  0.8764  0.8764  0.8728  0.0045  
MAE (testset)     0.6726  0.6739  0.6646  0.6718  0.6708  0.6707  0.0032  
Fit time          1.67    1.57    2.29    1.73    1.57    1.77    0.27    
Test time         0.25    0.12    0.20    0.23    0.11    0.18    0.06    


{'test_rmse': array([0.87164765, 0.87493315, 0.8643788 , 0.87643603, 0.87637262]),
 'test_mae': array([0.6725878 , 0.67394297, 0.66461894, 0.67183649, 0.67075951]),
 'fit_time': (1.6728947162628174,
  1.5743987560272217,
  2.2867603302001953,
  1.7268469333648682,
  1.57063627243042),
 'test_time': (0.2504262924194336,
  0.11761927604675293,
  0.2040550708770752,
  0.2324211597442627,
  0.11347818374633789)}

In [ ]:
# Predict the rating for a specific user and movie
user_id = 1  # User ID
movie_id = 10  # Movie ID

prediction = algo.predict(user_id, movie_id)
print(f"Predicted rating for user {user_id} and movie {movie_id}: {prediction.est}")


Predicted rating for user 1 and movie 10: 4.134846034427684


In [ ]:
# Get a list of all movie IDs
all_movie_ids = movies_data['movieId'].unique()

# Predict ratings for all movies that the user hasn't rated yet
user_ratings = {movie_id: algo.predict(user_id, movie_id).est for movie_id in all_movie_ids}

# Sort the movies by predicted rating in descending order
recommended_movies = sorted(user_ratings.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the top 10 recommended movies
print("Top 10 recommended movies for user", user_id)
for movie_id, rating in recommended_movies:
    movie_title = movies_data[movies_data['movieId'] == movie_id]['title'].values[0]
    print(f"{movie_title}: {rating}")


Top 10 recommended movies for user 1
Twelve Monkeys (a.k.a. 12 Monkeys) (1995): 5.0
Usual Suspects, The (1995): 5.0
Bottle Rocket (1996): 5.0
Pulp Fiction (1994): 5.0
Shawshank Redemption, The (1994): 5.0
In the Name of the Father (1993): 5.0
Fargo (1996): 5.0
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964): 5.0
Godfather, The (1972): 5.0
Casablanca (1942): 5.0
